<a href="https://colab.research.google.com/github/LeQuangHuyUIT/CS117.L21.KHCL/blob/main/Demo_CS117_L21_KHCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# git clone 

In [1]:
!git clone https://github.com/LeQuangHuyUIT/CS117.L21.KHCL

Cloning into 'CS117.L21.KHCL'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 2), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [2]:
%cd CS117.L21.KHCL/

/content/CS117.L21.KHCL


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 11.4 MB 5.4 MB/s 
     |████████████████████████████████| 100.1 MB 8.0 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=5f2b5a297e58d5af2ecf78459f890a0c8055cb515f7982c15d34a11bc5045dd7
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [4]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from dbr import *
import cv2

# Open and capture image

In [5]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import random

In [6]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [7]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data


In [8]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# Function for Face detection

In [110]:
path = '/content/CS117.L21.KHCL/faces'
images = []
classNames = []
myList = os.listdir(path)

for cl in myList:
    if os.path.splitext(cl)[1] == '.jpg':
        curImg = cv2.imread(f'{path}/{cl}')
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])
print(len(images))

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        face = face_recognition.face_locations(img)
        encode = face_recognition.face_encodings(img, face)[0]
        encodeList.append(encode)
    return encodeList
  
encodeListKnown = findEncodings(images)
print(len(encodeListKnown))
print('Encoding Complete')

6
6
Encoding Complete


In [111]:
print(os.path.splitext(myList[3]))

('donald trump', '.jpg')


In [112]:
def draw_face_BoudingBox(img):
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    name = ''

    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        name = 'Unknown'
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex] > 0.8:
            name = classNames[matchIndex].upper()
            #print(name)
        y1,x2,y2,x1 = faceLoc
        y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4

        img = cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
        img = cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
        img = cv2.putText(img,name,(x1+6,y2-6),\
                cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

        bbox_array = cv2.rectangle(bbox_array,(x1,y1),(x2,y2),(0,255,0),2)
        bbox_array = cv2.rectangle(bbox_array,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
        bbox_array = cv2.putText(bbox_array,name,(x1+6,y2-6),\
                cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

    return img, bbox_array, name

In [113]:
# start streaming video from webcam
def capture_and_face_identify():
    video_stream()
    # label for video
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    count = 0 
    new_images = []
    new_names = []
    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break

        # convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])

        # draw face bouding box
        detected_img, bbox_array, name = draw_face_BoudingBox(img.copy())

        bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
        # convert overlay of bbox into bytes
        bbox_bytes = bbox_to_bytes(bbox_array)
        # update bbox so next frame gets new overlay
        bbox = bbox_bytes

        if name == 'Unknown':
            new_name = 'New_face' + str(random.randint(1, 1000))
            while new_name in myList:
                new_name = 'New_face' + str(random.randint(1, 1000))

            imgS = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            facesCurFrame = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
            if len(encodesCurFrame):
                classNames.append(new_name)
                encodeListKnown.append(encodesCurFrame[0])
                new_path = os.path.join(path, new_name + '.jpg')
                new_images.append(img)
                new_names.append(new_path)

    plt.figure(figsize=(10, 20))
    for i, img in enumerate(new_images):
        cv2.imwrite(new_names[i], img)

# Function for barcode detection

In [12]:
license_key = "t0070fQAAAA/lgr6TSGYD/mYnKxOQQZdL5rCdz5VQaxSlgPnN4Fl+HhYfVISxWKOqL5dTllJaV8zwuNRTPk0MiyivIZVKR0Ia4g=="
reader = BarcodeReader()

# Apply for a trial license: https://www.dynamsoft.com/customer/license/trialLicense?product=dbr&utm_source=github
reader.init_license(license_key)

def detect_and_decode(image):

    text_results = reader.decode_buffer(image)
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    text = ""
    if text_results != None:
        for text_result in text_results:
            temp = text_result.localization_result.localization_points
            contour = np.array([list(x) for x in temp])

            font = cv2.FONT_HERSHEY_SIMPLEX
            org = (00, 185)
            fontScale = 0.5
            color = (0, 0, 255)
            thickness = 1
            text = text_result.barcode_text

            cv2.drawContours(image, [contour], -1, (0, 255, 0), 5)
            cv2.putText(image, text, (contour[0][0], contour[0][1]), font, fontScale, 
                color, thickness, cv2.LINE_AA, False)

            cv2.drawContours(bbox_array, [contour], -1, (0, 255, 0), 5)
            cv2.putText(bbox_array, text, (contour[0][0], contour[0][1]), font, fontScale, 
                color, thickness, cv2.LINE_AA, False)
            
    return image, bbox_array, text 


In [13]:
def capture_image_and_detect_barcode():
    video_stream()
    # label for video
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    count = 0 
    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            cv2.imwrite('barcode.jpg', detected_img)
            break

        # convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])

        # draw barcode bouding box
        detected_img, bbox_array, text = detect_and_decode(img)

        bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
        # convert overlay of bbox into bytes
        bbox_bytes = bbox_to_bytes(bbox_array)
        # update bbox so next frame gets new overlay
        bbox = bbox_bytes
    print("text: ", text)


# Face identify demo

In [114]:
capture_and_face_identify()

<IPython.core.display.Javascript object>

<Figure size 720x1440 with 0 Axes>

In [ ]:
capture_image_and_detect_barcode()